Модуль для расчета алгоритма типа 'logistic' с фичами по товарам и без, с кроссвалидацией

In [ ]:
pip install lightfm

     |████████████████████████████████| 307kB 2.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707639 sha256=46f0cfdb82717cb6d7ebe626885137a9335434ff4e9d2b300e9abd6b84685180
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

DIR = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/recommendationsv4/'
DIR_TEST = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/TEST/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm


In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

from lightfm.data import Dataset

In [ ]:
#Читаем файлы
data = pd.read_csv(DIR + 'train.csv',low_memory=False )
test = pd.read_csv(DIR +'test.csv',low_memory=False )
submission = pd.read_csv(DIR +'sample_submission.csv')

In [ ]:
# Определяем выбросы, вероятно спам
def get_spam(field):
  med = data[field].value_counts().describe()['50%']
  std = data[field].value_counts().describe()['std']
  counts = pd.DataFrame(data[field].value_counts())
  spam = list(counts[counts[field] > med + 3 * std].index)
  return spam

In [ ]:
item_spam = get_spam('itemid')
user_spam = get_spam('userid')
data = data[~((data['itemid'].isin(item_spam)) & (data['userid'].isin(user_spam)))]

In [ ]:
# Упорядочим наши данные по верифицированности ответа и времени
data.sort_values(by = ['verified', 'unixReviewTime'], ascending= True, inplace = True)

In [ ]:
#Удалим дубли, оставим только по одной паре  пользователь товар, 
#оставим последний отзыв,считая, что  могут меняться вкусы и качество товаров
data.drop_duplicates(subset =['userid','unixReviewTime','itemid'], keep = 'last',inplace = True)

In [ ]:
len(data), len(test),  len(test)/len(data)

(806942, 285965, 0.35438110793588634)

In [ ]:
data = data.reset_index()

In [ ]:
# Проверим , что между  asin и itemid однозначное соответствие, что данные корректны
data_all = pd.concat([data, test])
data_all_test = pd.DataFrame(data_all.groupby(['asin']).itemid.max() - data_all.groupby(['asin']).itemid.min())
data_all_test.itemid.value_counts()
# должен быть ноль по всей длине датафрейма

0    41320
Name: itemid, dtype: int64

In [ ]:
#Словарь соответствия asin и itemid
mapper = dict(zip(data_all.asin, data_all.itemid))
len(mapper)

41320

In [ ]:
import json
def get_by_key(field, dict_from, dict_to):
   if field in dict_from.keys():
       dict_to[mapper.get(dict_from.get('asin'))] = dict_from.get(field)    

with open(DIR + 'meta_Grocery_and_Gourmet_Food_small.json') as file:
     d_main_category = {}
     d_brand = {}
     d_category = {}
     for line in file.readlines(): 
         lines = json.loads(line) 
         get_by_key('brand', lines, d_brand)
         get_by_key('main_cat', lines, d_main_category)
         get_by_key('category', lines, d_category)

In [ ]:
set_itemid = set(data['itemid']) | set(test['itemid']) 
#Подготовим словари с фичами для модели, обеспечим уникальность значения
d_brand_ml =  {x:'brand:' + y for x,y in d_brand.items()}
d_main_category_ml = {x:'main_cat:' + y for x,y in d_main_category.items()}
#Поиграем самой категорией, возьмем самый нижний уровень
d_last_category_ml = {x:'last_cat:' + y[-1] for x,y in d_category.items()}

In [ ]:
#Формируем  mapping для всех пользователей и товаров(train + test)

dataset = Dataset() #item_identity_features=False)
dataset.fit((x for x in pd.concat([data, test])['userid']),
            (x for x in pd.concat([data, test])['itemid']))

In [ ]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 127496, num_items 41320.


In [ ]:
# Добавляем фичи по товарам 
dataset.fit_partial(items=(x for x in d_last_category_ml.keys()),
                    item_features=(x for x in d_last_category_ml.values()))

dataset.fit_partial(items=(x for x in d_main_category_ml.keys()),
                    item_features=(x for x in d_main_category_ml.values()))

dataset.fit_partial(items=(x for x in d_brand_ml.keys()),
                    item_features=(x for x in d_brand_ml.values()))

In [ ]:
#Готовим основную матрицу
(interactions, weights) = dataset.build_interactions(((x,y,z) 
              #  for x,y, z in zip(train_data.userid.values,train_data.itemid.values, train_data.rating.values)))
                for x,y, z in zip(data.userid.values,data.itemid.values, data.rating.values)))

print(repr(interactions))

<127496x41320 sparse matrix of type '<class 'numpy.int32'>'
	with 806942 stored elements in COOrdinate format>


In [ ]:
# Готовим фичи по товарам
def get_item_features(itemid):
   def add_from_dict(dict_):
      if dict_.get(itemid):
         list_feat.append(dict_.get(itemid)) 

   list_feat =[]
   add_from_dict(d_brand_ml)
   add_from_dict(d_last_category_ml)
   add_from_dict(d_main_category_ml)
   return list_feat

item_features = dataset.build_item_features(((x, get_item_features(x))
                                               for x in set_itemid ))
print(repr(item_features))

<41320x51173 sparse matrix of type '<class 'numpy.float32'>'
	with 164772 stored elements in Compressed Sparse Row format>


In [ ]:
# Вытаскиваем словари для преобразований
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [ ]:
from lightfm import LightFM
NUM_THREADS = 8 #число потоков
RANDOM_SEED = 32
#Гиперпараметры
#NUM_EPOCHS, LEARNING_RATE, NUM_COMPONENTS, ALPHA =  [108,0.9307615113531499,156,1.117646374973343e-06]
#params = [15,0.1,120,0]  #стартовые параметры
params_not_features = [108,0.9307615113531499,156,1.117646374973343e-06] # без фич
params =  [54, 0.6695553384725956, 90, 4.527672167401107e-06] #с фичами
#params = [119, 0.6357799002551178, 97, 4.8911903142209495e-06]

In [ ]:
def prediction_model(mtype, h_params, mat_weights):
    num_epochs, learning_rate, num_components, alpha = h_params
    model = LightFM(learning_rate = learning_rate, 
                          loss = mtype, 
                          no_components = num_components,
                          random_state = RANDOM_SEED,
                          user_alpha = alpha,
                          item_alpha = alpha)
    model = model.fit(mat_weights, epochs=num_epochs, num_threads=NUM_THREADS
                              , item_features=item_features
                              ) 
    
    return(model)


N_FOLDS = 5
sample_submissions = pd.DataFrame(0,columns=["sub_1"], index=submission.index) # куда пишем предикты по каждой модели
score_ls = []
X = data[['userid','itemid']]
y = data.rating
#splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

splits = list(StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))


for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # Строим разреженную матрицу 
#Готовим основную матрицу
    (interactions, weights_train) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(X_train.userid.values, X_train.itemid.values, y_train)))
    # model for this fold
    model_light = prediction_model('logistic', params, weights_train)

    # score model on test
    test_predict = model_light.predict(X_test.userid.apply(lambda x: user_id_map[x]).values,
                              X_test.itemid.apply(lambda x: item_id_map[x]).values 
                             ,item_features=item_features
                             )

    test_score = sklearn.metrics.roc_auc_score(y_test,test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {test_score:0.4f}")
    # submissions
    sample_submissions[f'sub_{idx+1}'] = model_light.predict(test.userid.apply(lambda x: user_id_map[x]).values,
                                                             test.itemid.apply(lambda x: item_id_map[x]).values
                                                             ,item_features=item_features 
                                                              )
    
print(f'Mean Score: {np.mean(score_ls):0.5f}')
print(f'Std Score: {np.std(score_ls):0.5f}')
print(f'Max Score: {np.max(score_ls):0.5f}')
print(f'Min Score: {np.min(score_ls):0.5f}')    

1 Fold Test MAPE: 0.7399
2 Fold Test MAPE: 0.7406
3 Fold Test MAPE: 0.7369
4 Fold Test MAPE: 0.7373
5 Fold Test MAPE: 0.7406

Mean Score: 0.73905
Std Score: 0.00163
Max Score: 0.74062
Min Score: 0.73691


In [ ]:
sample_submissions['res'] = sample_submissions.mean(axis=1)
sample_submissions

,sub_1,sub_2,sub_3,sub_4,sub_5,res
0,0.398405,0.429004,0.407645,0.411623,0.417448,0.412825
1,0.471967,0.416494,0.411615,0.368333,0.430257,0.419733
2,0.335603,0.512230,0.361715,0.344687,0.373546,0.385556
3,0.297580,0.293909,0.368526,0.334036,0.354550,0.329720
4,0.349095,0.195303,0.225379,0.179776,0.152137,0.220338
...,...,...,...,...,...,...
285960,0.463908,0.458010,0.449312,0.499049,0.478608,0.469777
285961,-0.177673,-0.132821,-0.039137,-0.082233,0.398131,-0.006746
285962,0.355145,0.254361,0.245107,0.267172,0.237769,0.271911
285963,0.348215,0.335280,0.315816,0.510952,0.353326,0.372718


In [ ]:
preds = sample_submissions['res']

In [ ]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [ ]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [ ]:
submission['rating']= normalized_preds

In [ ]:
VERSION = 19
submission.to_csv(DIR_TEST+ f'submission_v{VERSION}.csv', index=False)


In [ ]:
submission

,Id,rating
0,0,0.569686
1,1,0.574666
2,2,0.550029
3,3,0.509779
4,4,0.430930
...,...,...
285960,285960,0.610741
285961,285961,0.267233
285962,285962,0.468106
285963,285963,0.540774


Вроде не выдающийся результат, но на каггле дал 0.77689

In [ ]:
def prediction_model_not_features(mtype, h_params, mat_weights):
    num_epochs, learning_rate, num_components, alpha = h_params
    model = LightFM(learning_rate = learning_rate, 
                          loss = mtype, 
                          no_components = num_components,
                          random_state = RANDOM_SEED,
                          user_alpha = alpha,
                          item_alpha = alpha)
    model = model.fit(mat_weights, epochs=num_epochs, num_threads=NUM_THREADS)    
    return(model)


N_FOLDS = 5
sample_submissions_not_features = pd.DataFrame(0,columns=["sub_1"], index=submission.index) # куда пишем предикты по каждой модели
score_ls_not_features = []
X = data[['userid','itemid']]
y = data.rating
#splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

splits = list(StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))


for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # Строим разреженную матрицу 
#Готовим основную матрицу
    (interactions, weights_train) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(X_train.userid.values, X_train.itemid.values, y_train)))
    # model for this fold
    model_light_not_features = prediction_model_not_features('logistic', params_not_features, weights_train)

    # score model on test
    test_predict = model_light_not_features.predict(X_test.userid.apply(lambda x: user_id_map[x]).values, X_test.itemid.apply(lambda x: item_id_map[x]).values)

    test_score = sklearn.metrics.roc_auc_score(y_test,test_predict)
    score_ls_not_features.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {test_score:0.4f}")
    # submissions
    sample_submissions_not_features[f'sub_{idx+1}'] = model_light_not_features.predict(test.userid.apply(lambda x: user_id_map[x]).values, test.itemid.apply(lambda x: item_id_map[x]).values)

 #   model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls_not_features):0.5f}')
print(f'Std Score: {np.std(score_ls_not_features):0.5f}')
print(f'Max Score: {np.max(score_ls_not_features):0.5f}')
print(f'Min Score: {np.min(score_ls_not_features):0.5f}')    

1 Fold Test MAPE: 0.7303
2 Fold Test MAPE: 0.7325
3 Fold Test MAPE: 0.7345
4 Fold Test MAPE: 0.7350
5 Fold Test MAPE: 0.7330

Mean Score: 0.73306
Std Score: 0.00166
Max Score: 0.73501
Min Score: 0.73032


In [ ]:
sample_submissions_not_features['res'] = sample_submissions_not_features.mean(axis=1)
sample_submissions_not_features

,sub_1,sub_2,sub_3,sub_4,sub_5,res
0,1.872681,1.970547,1.857385,1.820142,1.794750,1.863101
1,1.907098,1.525849,0.786301,0.521529,1.279796,1.204115
2,0.226045,0.944301,0.038497,0.168206,-0.299550,0.215500
3,0.050269,0.331990,0.417444,0.270822,0.452952,0.304695
4,2.019630,1.446576,1.192284,0.533416,1.575157,1.353413
...,...,...,...,...,...,...
285960,2.019759,2.011352,1.564522,1.601647,2.112386,1.861933
285961,-1.883562,-0.999788,-1.837296,-1.713308,-0.138489,-1.314489
285962,0.861335,0.860885,-0.180254,0.985273,-0.201969,0.465054
285963,1.818834,1.512112,1.408063,2.121469,2.259250,1.823946


In [ ]:
preds = sample_submissions_not_features['res']
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [ ]:
submission['rating']= normalized_preds

In [ ]:
submission

,Id,rating
0,0,0.647851
1,1,0.572375
2,2,0.459145
3,3,0.469361
4,4,0.589474
...,...,...
285960,285960,0.647717
285961,285961,0.283911
285962,285962,0.487728
285963,285963,0.643366


In [ ]:
VERSION = 18
submission.to_csv(DIR_TEST+ f'submission_v{VERSION}.csv', index=False)